In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pickle
file_to_read = open("/content/drive/MyDrive/NLP_project/X_train_final_anupam", "rb")
X_train = pickle.load(file_to_read)
file_to_read = open("/content/drive/MyDrive/NLP_project/y_train_final_anupam", "rb")
y_train = pickle.load(file_to_read)

In [3]:
X_train.shape

(109770, 60, 100)

In [4]:
y_train.shape

(109770, 1, 3)

In [5]:
import tensorflow as tf
import keras 
title_input = keras.Input(
    shape=(60,100), name="title"
)
y1=keras.layers.Conv1D(40,2,activation='relu',input_shape=(60,100))(title_input)
#y2=keras.layers.Conv1D(32,2,activation='relu',input_shape=(100,100))(title_input)
y3=keras.layers.Conv1D(40,3,activation='relu',input_shape=(60,100))(title_input)
#y4=keras.layers.Conv1D(32,3,activation='relu',input_shape=(60,100))(title_input)
y5=keras.layers.Conv1D(40,4,activation='relu',input_shape=(60,100))(title_input)
y6=keras.layers.Conv1D(40,5,activation='relu',input_shape=(60,100))(title_input)

m1=tf.keras.layers.MaxPooling1D(pool_size=59,strides=1)(y1)
#m2=tf.keras.layers.MaxPooling1D(pool_size=59,strides=1)(y2)
m3=tf.keras.layers.MaxPooling1D(pool_size=58,strides=1)(y3)
#m4=tf.keras.layers.MaxPooling1D(pool_size=58,strides=1)(y4)
m5=tf.keras.layers.MaxPooling1D(pool_size=57,strides=1)(y5)
m6=tf.keras.layers.MaxPooling1D(pool_size=56,strides=1)(y6)

#concanate = tf.keras.layers.concatenate([m1,m2,m3,m4,m5,m6])
concanate = tf.keras.layers.concatenate([m1,m3,m5,m6])
#den_lay1 = tf.keras.layers.Dense(256, name="MLP1")(concanate)

#den_lay2 = tf.keras.layers.Dense(128, name="MLP2")(den_lay1)

den_lay3 = tf.keras.layers.Dense(64, name="MLP3")(concanate)

den_lay4 = tf.keras.layers.Dense(32, name="MLP4")(den_lay3)

den_lay5 = tf.keras.layers.Dense(16, name="MLP5")(den_lay4)

op_lay = tf.keras.layers.Dense(3, activation='softmax',name="priority")(den_lay5)

model = keras.Model(
    inputs=[title_input],
    outputs=[op_lay],
)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
model.fit(X_train[:90000],y_train[:90000],validation_split=0.1,epochs=10)

Epoch 1/10
2532/2532 [==============================] - 47s 15ms/step - loss: 0.8808 - accuracy: 0.5887 - val_loss: 0.8004 - val_accuracy: 0.6282
Epoch 2/10
2532/2532 [==============================] - 36s 14ms/step - loss: 0.7772 - accuracy: 0.6493 - val_loss: 0.8637 - val_accuracy: 0.6129
Epoch 3/10
2532/2532 [==============================] - 36s 14ms/step - loss: 0.7307 - accuracy: 0.6770 - val_loss: 0.8040 - val_accuracy: 0.6371
Epoch 4/10
2532/2532 [==============================] - 36s 14ms/step - loss: 0.6907 - accuracy: 0.6976 - val_loss: 0.7708 - val_accuracy: 0.6461
Epoch 5/10
2532/2532 [==============================] - 36s 14ms/step - loss: 0.6521 - accuracy: 0.7186 - val_loss: 0.7847 - val_accuracy: 0.6460
Epoch 6/10
2532/2532 [==============================] - 36s 14ms/step - loss: 0.6135 - accuracy: 0.7370 - val_loss: 0.8764 - val_accuracy: 0.6241
Epoch 7/10
2532/2532 [==============================] - 36s 14ms/step - loss: 0.5820 - accuracy: 0.7511 - val_loss: 0.8560 -

In [9]:
import tensorflow as tf
y_pred=model.predict(X_train[90000:])
#con_mat = tf.math.confusion_matrix(labels=y_train[90000:], predictions=y_pred).numpy()

In [10]:
y_pred.shape

(19770, 1, 3)

In [17]:
y_pred[0][0]

array([0.35753807, 0.49510047, 0.14736149], dtype=float32)

In [18]:
for i in range(y_pred.shape[0]):
  if(y_pred[i][0][0]>y_pred[i][0][1] and y_pred[i][0][0]>y_pred[i][0][2]):
    y_pred[i][0][0]=1
    y_pred[i][0][1]=0
    y_pred[i][0][2]=0
  elif(y_pred[i][0][1]>y_pred[i][0][0] and y_pred[i][0][1]>y_pred[i][0][2]):
    y_pred[i][0][1]=1
    y_pred[i][0][0]=0
    y_pred[i][0][2]=0
  else:
    y_pred[i][0][2]=1
    y_pred[i][0][0]=0
    y_pred[i][0][1]=0

In [19]:
matrix=[[0,0,0],[0,0,0],[0,0,0]]
for i in range(y_pred.shape[0]):
  if(y_train[90000+i][0][0]==1):
    if(y_pred[i][0][0]==1):
      matrix[0][0]+=1
    if(y_pred[i][0][1]==1):
      matrix[0][1]+=1
    if(y_pred[i][0][2]==1):
      matrix[0][2]+=1
  elif(y_train[90000+i][0][1]==1):
    if(y_pred[i][0][0]==1):
      matrix[1][0]+=1
    if(y_pred[i][0][1]==1):
      matrix[1][1]+=1
    if(y_pred[i][0][2]==1):
      matrix[1][2]+=1
  else:
    if(y_pred[i][0][0]==1):
      matrix[2][0]+=1
    if(y_pred[i][0][1]==1):
      matrix[2][1]+=1
    if(y_pred[i][0][2]==1):
      matrix[2][2]+=1

In [20]:
matrix

[[5052, 1007, 1162], [1640, 1704, 1935], [808, 596, 5866]]

In [ ]:
import numpy as np
import pandas as pd
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)

con_mat_df = pd.DataFrame(con_mat_norm,
                     index = classes, 
                     columns = classes)

In [ ]:
import matplotlib.pyplot as plt
figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
model.pre